In [2]:
import numpy as np
import pickle
import mne
from models.EEGModels import EEGNet

import torch
from braindecode.models import EEGNetv4, EEGConformer, ATCNet, EEGITNet, EEGInception
from skorch import NeuralNetClassifier
from models import cuda
from skorch.callbacks import LRScheduler, TrainEndCheckpoint

from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K


from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import os
import visualkeras

# Set image data format
K.set_image_data_format('channels_last')

import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

c:\temp\BME_442\bme442project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
c:\temp\BME_442\bme442project\.venv\lib\site-packages\moabb\pipelines\__init__.py:26

In [3]:
# Define the base directory
base_dir = './preprocessed_data/'

# Initialize lists to hold the combined data
X_combined = []
y_combined = []

# with open('C:\\temp\\BME_442\\bme442project\\preprocessed_data\S001\\task_1\\run_3.pkl', 'rb') as f:
#     data = pickle.load(f)
#     print(data['y'])

# Load data from each subject
for subject in os.listdir(base_dir):
    subject_dir = os.path.join(base_dir, subject)
    if os.path.isdir(subject_dir):
        for task in ['task_1', 'task_2', 'task_3', 'task_4']:
            task_dir = os.path.join(subject_dir, task)
            if os.path.isdir(task_dir):
                for run_file in os.listdir(task_dir):
                    if run_file.endswith('.pkl'):
                        run_path = os.path.join(task_dir, run_file)
                        print(run_path)
                        with open(run_path, 'rb') as f:
                            data = pickle.load(f)
                            X_combined.append(data['X'])
                            y_combined.append(data['y'])
                            # print(X_combined)


./preprocessed_data/S001\task_1\run_11.pkl
./preprocessed_data/S001\task_1\run_3.pkl
./preprocessed_data/S001\task_1\run_7.pkl
./preprocessed_data/S001\task_2\run_12.pkl
./preprocessed_data/S001\task_2\run_4.pkl
./preprocessed_data/S001\task_2\run_8.pkl
./preprocessed_data/S001\task_3\run_13.pkl
./preprocessed_data/S001\task_3\run_5.pkl
./preprocessed_data/S001\task_3\run_9.pkl
./preprocessed_data/S001\task_4\run_10.pkl
./preprocessed_data/S001\task_4\run_14.pkl
./preprocessed_data/S001\task_4\run_6.pkl
./preprocessed_data/S002\task_1\run_11.pkl
./preprocessed_data/S002\task_1\run_3.pkl
./preprocessed_data/S002\task_1\run_7.pkl
./preprocessed_data/S002\task_2\run_12.pkl
./preprocessed_data/S002\task_2\run_4.pkl
./preprocessed_data/S002\task_2\run_8.pkl
./preprocessed_data/S002\task_3\run_13.pkl
./preprocessed_data/S002\task_3\run_5.pkl
./preprocessed_data/S002\task_3\run_9.pkl
./preprocessed_data/S002\task_4\run_10.pkl
./preprocessed_data/S002\task_4\run_14.pkl
./preprocessed_data/S002

In [4]:

# Convert lists to arrays
y_combined = np.concatenate(y_combined, axis=0)
X_combined = np.concatenate(X_combined, axis=0)

# Check the shapes of the combined datasets
print("X_combined shape:", X_combined.shape)
print("y_combined shape:", y_combined.shape)


X_combined shape: (38383, 64, 656)
y_combined shape: (38383,)


In [5]:
unique, counts = np.unique(y_combined, return_counts=True)
print(dict(zip(unique, counts)))

{1: 19674, 2: 4697, 3: 4662, 4: 4668, 5: 4682}


In [6]:

# Ensure that the number of samples is correct for y
if X_combined.shape[0] != y_combined.shape[0]:
    raise ValueError("Mismatch between number of samples in X_combined and y_combined.")


In [7]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [9]:
callbacks = [("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=200 - 1))]

model = NeuralNetClassifier(module=EEGNetv4,
                                       module__n_chans=64,
                                       module__n_outputs=5,
                                       criterion=torch.nn.CrossEntropyLoss,
                                       optimizer=torch.optim.Adam,
                                       optimizer__lr=0.1,
                                       iterator_train__shuffle=True,
                                       callbacks=callbacks,
                                       device='cuda' if cuda else 'cpu',
                                       verbose=True
                                       )

In [24]:
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
import concurrent.futures

In [25]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [27]:
def process_fold(train_index, test_index):
    train_dataset = CustomDataset(X_combined[train_index], y_combined[train_index])
    test_dataset = CustomDataset(X_combined[test_index], y_combined[test_index])
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    # Fit the neural network
    model.fit(train_loader, y=None)  # Passing DataLoader directly

    # Predict and evaluate
    y_pred = []
    y_true = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_batch_pred = model.predict(X_batch)
            y_pred.extend(y_batch_pred)
            y_true.extend(y_batch)
    
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Fold accuracy: {accuracy}')
    
    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print(f'Confusion Matrix:\n{cm}')
    
    # Calculate sensitivities, specificities, ppvs, npvs
    fold_sensitivities = []
    fold_specificities = []
    fold_ppvs = []
    fold_npvs = []
    
    for i in range(5):  # Assuming 5 classes
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - (tp + fn + fp)
        
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
        npv = tn / (tn + fn) if (tn + fn) > 0 else 0
        
        fold_sensitivities.append(sensitivity)
        fold_specificities.append(specificity)
        fold_ppvs.append(ppv)
        fold_npvs.append(npv)

    return fold_sensitivities, fold_specificities, fold_ppvs, fold_npvs


In [28]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [29]:
sensitivities = []
specificities = []
ppvs = []
npvs = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for train_index, test_index in skf.split(X_combined, y_combined):
        futures.append(executor.submit(process_fold, train_index, test_index))
    
    for future in concurrent.futures.as_completed(futures):
        fold_sensitivities, fold_specificities, fold_ppvs, fold_npvs = future.result()
        sensitivities.extend(fold_sensitivities)
        specificities.extend(fold_specificities)
        ppvs.extend(fold_ppvs)
        npvs.extend(fold_npvs)

# Calculate average metrics
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)
avg_ppv = np.mean(ppvs)
avg_npv = np.mean(npvs)

print(f'Average Sensitivity: {avg_sensitivity}')
print(f'Average Specificity: {avg_specificity}')
print(f'Average PPV: {avg_ppv}')
print(f'Average NPV: {avg_npv}')


MemoryError: Unable to allocate 9.61 GiB for an array with shape (30707, 64, 656) and data type float64

In [ ]:

# Calculate average metrics
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)
avg_ppv = np.mean(ppvs)
avg_npv = np.mean(npvs)

print(f'Average Sensitivity: {avg_sensitivity}')
print(f'Average Specificity: {avg_specificity}')
print(f'Average PPV: {avg_ppv}')
print(f'Average NPV: {avg_npv}')


In [ ]:

# Reshape data for EEGNet
kernels, chans, samples = 1, X_train.shape[1], X_train.shape[2]

X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_validate = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

print('Y_train shape', Y_train.shape)
print('Y_test shape', Y_test.shape)



In [ ]:
# Initialize EEGNet model
model = EEGNet(nb_classes=Y_train.shape[1], Chans=chans, Samples=samples,
               dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16,
               dropoutType='Dropout')

visualkeras.layered_view(model, to_file='model.png').show()

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up model checkpointing
checkpointer = ModelCheckpoint(filepath='./tmp/checkpoint.h5', verbose=1, save_best_only=True)

# Train the model
fittedModel = model.fit(X_train, Y_train, batch_size=16, epochs=300, verbose=2,
                        validation_data=(X_validate, Y_validate),
                        callbacks=[checkpointer])


In [ ]:

# Load best weights
model.load_weights('./tmp/checkpoint.h5')

# Print checkpoint folder full path
print("Model checkpoint folder:", os.path.abspath('./tmp/checkpoint.h5'))

# Evaluate model
probs = model.predict(X_test)
preds = probs.argmax(axis=-1)
acc = np.mean(preds == Y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

# Plot confusion matrices
names = ['rest', 'left fist', 'right fist', 'both fists', 'both feet']

cm_eegnet = confusion_matrix(Y_test.argmax(axis=-1), preds)
disp_eegnet = ConfusionMatrixDisplay(confusion_matrix=cm_eegnet, display_labels=names)
disp_eegnet.plot(cmap=plt.cm.Blues)
plt.title('EEGNet-8,2')
plt.show()
